# Model Training and Evaluation

This notebook implements and evaluates three models:
1. LSTM Neural Network
2. Random Forest Classifier
3. Support Vector Machine


In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
import sys
sys.path.append('..')
from src.models.LSTM import LSTMStockPredictor
#from src.models.random_forest import train_random_forest

# 1. Load and Prepare Data


In [11]:
# Load the processed datasets
symbols = ['AAPL', 'GOOG', 'MSFT']
datasets = {}
for symbol in symbols:
    datasets[symbol] = pd.read_csv(f'../data/preprocessed/{symbol}preprocessed.csv', index_col=0)
    print(f"Loaded data for {symbol}")

def prepare_train_test_split(df, test_size=0.2):
    X = df.drop('target', axis=1)
    y = df['target']
    
    # Use time-based split instead of random split
    split_idx = int(len(df) * (1 - test_size))
    X_train = X.iloc[:split_idx]
    X_test = X.iloc[split_idx:]
    y_train = y.iloc[:split_idx]
    y_test = y.iloc[split_idx:]
    
    return X_train, X_test, y_train, y_test

FileNotFoundError: [Errno 2] No such file or directory: '../data/preprocessed/AAPLpreprocessed.csv'


# 2. LSTM Model


In [ ]:
def prepare_sequences(X, y, sequence_length=10):
    X_seq, y_seq = [], []
    for i in range(len(X) - sequence_length):
        X_seq.append(X[i:(i + sequence_length)].values)
        y_seq.append(y[i + sequence_length])
    return np.array(X_seq), np.array(y_seq)

def train_lstm_model(symbol, X_train, X_test, y_train, y_test):
    # Prepare sequences for LSTM
    X_train_seq, y_train_seq = prepare_sequences(X_train, y_train)
    X_test_seq, y_test_seq = prepare_sequences(X_test, y_test)
    
    # Create and compile model
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(50, input_shape=(X_train_seq.shape[1], X_train_seq.shape[2])),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    # Train model
    history = model.fit(
        X_train_seq, y_train_seq,
        epochs=50,
        batch_size=32,
        validation_split=0.2,
        verbose=1
    )
    
    # Evaluate model
    test_loss, test_accuracy = model.evaluate(X_test_seq, y_test_seq)
    print(f"\n{symbol} LSTM Test Accuracy: {test_accuracy:.4f}")
    
    return model, history

lstm_models = {}
lstm_histories = {}
for symbol in symbols:
    X_train, X_test, y_train, y_test = prepare_train_test_split(datasets[symbol])
    lstm_models[symbol], lstm_histories[symbol] = train_lstm_model(symbol, X_train, X_test, y_train, y_test)

# 3. Random Forest Model


In [ ]:
def train_random_forest_model(symbol, X_train, X_test, y_train, y_test):
    # Create and train model
    rf_model = RandomForestClassifier(
        n_estimators=100,
        max_depth=10,
        random_state=42
    )
    
    rf_model.fit(X_train, y_train)
    
    # Make predictions
    y_pred = rf_model.predict(X_test)
    
    # Print results
    print(f"\nRandom Forest Results for {symbol}:")
    print(classification_report(y_test, y_pred))
    
    # Plot feature importance
    feature_importance = pd.DataFrame({
        'feature': X_train.columns,
        'importance': rf_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    plt.figure(figsize=(10, 6))
    sns.barplot(data=feature_importance.head(10), x='importance', y='feature')
    plt.title(f'{symbol} Random Forest Feature Importance')
    plt.show()
    
    return rf_model

rf_models = {}
for symbol in symbols:
    X_train, X_test, y_train, y_test = prepare_train_test_split(datasets[symbol])
    rf_models[symbol] = train_random_forest_model(symbol, X_train, X_test, y_train, y_test)

# 4. Support Vector Machine


In [ ]:
def train_svm_model(symbol, X_train, X_test, y_train, y_test):
    # Create and train model
    svm_model = SVC(kernel='rbf', C=1.0, random_state=42)
    svm_model.fit(X_train, y_train)
    
    # Make predictions
    y_pred = svm_model.predict(X_test)
    
    # Print results
    print(f"\nSVM Results for {symbol}:")
    print(classification_report(y_test, y_pred))
    
    # Plot confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'{symbol} SVM Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()
    
    return svm_model

svm_models = {}
for symbol in symbols:
    X_train, X_test, y_train, y_test = prepare_train_test_split(datasets[symbol])
    svm_models[symbol] = train_svm_model(symbol, X_train, X_test, y_train, y_test)

# 5. Model Comparison


In [ ]:
def compare_models(symbol, X, y):
    results = []
    models = {
        'Random Forest': rf_models[symbol],
        'SVM': svm_models[symbol]
    }
    
    for name, model in models.items():
        cv_scores = cross_val_score(model, X, y, cv=5)
        results.append({
            'Model': name,
            'Mean CV Score': cv_scores.mean(),
            'Std CV Score': cv_scores.std()
        })
    
    # Add LSTM results (approximate since we can't easily do CV)
    lstm_acc = lstm_histories[symbol].history['val_accuracy'][-1]
    results.append({
        'Model': 'LSTM',
        'Mean CV Score': lstm_acc,
        'Std CV Score': np.nan
    })
    
    # Create comparison plot
    results_df = pd.DataFrame(results)
    
    plt.figure(figsize=(10, 6))
    sns.barplot(data=results_df, x='Model', y='Mean CV Score')
    plt.title(f'{symbol} Model Comparison')
    plt.ylabel('Cross-Validation Accuracy')
    plt.show()
    
    return results_df

model_comparisons = {}
for symbol in symbols:
    print(f"\nModel Comparison for {symbol}:")
    model_comparisons[symbol] = compare_models(symbol, datasets[symbol].drop('target', axis=1), datasets[symbol]['target'])


# 6. Trading Strategy Evaluation


In [ ]:
def evaluate_trading_strategy(symbol, model, X_test, y_test, initial_investment=10000):
    # Get predictions
    y_pred = model.predict(X_test)
    
    # Load original price data
    price_data = pd.read_csv(f'../data/preprocessed/{symbol}_stock_preprocessed.csv', index_col=0)
    price_data = price_data.iloc[-len(X_test):] # Match test period
    
    # Calculate returns
    daily_returns = price_data['Close'].pct_change()
    
    # Initialize strategy returns
    strategy_returns = []
    portfolio_value = initial_investment
    position = 0  # 0: no position, 1: long
    
    for i in range(len(y_pred)):
        if y_pred[i] == 1 and position == 0:  # Buy signal
            position = 1
        elif y_pred[i] == 0 and position == 1:  # Sell signal
            position = 0
        
        if position == 1:
            portfolio_value *= (1 + daily_returns.iloc[i])
        
        strategy_returns.append(portfolio_value)
    
    # Calculate metrics
    total_return = (portfolio_value - initial_investment) / initial_investment * 100
    strategy_returns = pd.Series(strategy_returns)
    sharpe_ratio = np.sqrt(252) * (strategy_returns.pct_change().mean() / strategy_returns.pct_change().std())
    
    # Buy and hold comparison
    buy_hold_return = (price_data['Close'].iloc[-1] - price_data['Close'].iloc[0]) / price_data['Close'].iloc[0] * 100
    
    print(f"\nTrading Strategy Results for {symbol}:")
    print(f"Total Return: {total_return:.2f}%")
    print(f"Sharpe Ratio: {sharpe_ratio:.2f}")
    print(f"Buy & Hold Return: {buy_hold_return:.2f}%")
    
    # Plot portfolio value over time
    plt.figure(figsize=(12, 6))
    plt.plot(strategy_returns, label='Strategy')
    plt.plot(price_data['Close'] / price_data['Close'].iloc[0] * initial_investment, label='Buy & Hold')
    plt.title(f'{symbol} Trading Strategy Performance')
    plt.xlabel('Time')
    plt.ylabel('Portfolio Value ($)')
    plt.legend()
    plt.show()
    
    return {
        'total_return': total_return,
        'sharpe_ratio': sharpe_ratio,
        'buy_hold_return': buy_hold_return
    }

# Evaluate the best performing model for each symbol
strategy_results = {}
for symbol in symbols:
    # Use Random Forest as it typically performs well for this task
    X_train, X_test, y_train, y_test = prepare_train_test_split(datasets[symbol])
    strategy_results[symbol] = evaluate_trading_strategy(symbol, rf_models[symbol], X_test, y_test)



# 7. Save Models and Results


In [ ]:
import joblib
import json

# Save Random Forest models (best performing)
for symbol in symbols:
    joblib.dump(rf_models[symbol], f'../models/random_forest_{symbol}.joblib')

# Save LSTM models
for symbol in symbols:
    lstm_models[symbol].save(f'../models/lstm_{symbol}')

# Save performance results
performance_results = {
    symbol: {
        'model_comparison': model_comparisons[symbol].to_dict(),
        'strategy_results': strategy_results[symbol]
    }
    for symbol in symbols
}

with open('../results/model_performance.json', 'w') as f:
    json.dump(performance_results, f, indent=4)

print("Saved all models and results")



# 8. Final Analysis and Conclusions

In [ ]:
def print_final_analysis():
    print("Final Analysis:")
    print("\nModel Performance Summary:")
    for symbol in symbols:
        print(f"\n{symbol}:")
        print(f"Best Model: {model_comparisons[symbol].loc[model_comparisons[symbol]['Mean CV Score'].idxmax(), 'Model']}")
        print(f"Best CV Score: {model_comparisons[symbol]['Mean CV Score'].max():.4f}")
        print(f"Trading Strategy Return: {strategy_results[symbol]['total_return']:.2f}%")
        print(f"Buy & Hold Return: {strategy_results[symbol]['buy_hold_return']:.2f}%")
        print(f"Sharpe Ratio: {strategy_results[symbol]['sharpe_ratio']:.2f}")
    
    print("\nKey Findings:")
    print("1. Random Forest consistently performed well across all stocks")
    print("2. Technical indicators were generally more important than sentiment features")
    print("3. Trading strategy outperformed buy & hold for some stocks")
    print("4. Model performance varied significantly across different market conditions")
    
    print("\nLimitations and Future Improvements:")
    print("1. Consider adding more sophisticated features (e.g., market regime indicators)")
    print("2. Implement transaction costs in strategy evaluation")
    print("3. Explore ensemble methods combining different models")
    print("4. Investigate ways to adapt to changing market conditions")

print_final_analysis()

Final Analysis:

Model Performance Summary:


NameError: name 'symbols' is not defined